# XGBoost

就差一个特征值为real类型的regression数据集来验证性能了

## XGBoost的regression版本, 损失函数为平方损失

In [1]:
import numpy as np

In [2]:
def tree_predict(x, tree):
    if x[tree['split_feat']] <= tree['split_val']:
        if type(tree['left']).__name__ == 'dict':
            return tree_predict(x, tree['left'])
        else:
            return tree['left']
        
    else:
        if type(tree['right']).__name__ == 'dict':
            return tree_predict(x, tree['right'])
        else:
            return tree['right']

In [3]:
def XGBoost_predict(X, previous_trees):
    result = []
    for i in range(X.shape[0]):
        result.append(0)
        for tree in previous_trees:
            result[-1] += tree_predict(X[i], tree)
            
    return np.array(result)

In [4]:
def loss_fun(y, y_hat):
    return 0.5 * ((y - y_hat) **2)

def calGain(dataSet, tree_list, loss_fun=None, lambda_=1):
    y_hat = XGBoost_predict(dataSet[:, :-1], tree_list)
    G = (y_hat - dataSet[:, -1]).sum()
    H = dataSet.shape[0]    #二阶导为1, 所以 H == 样本数
    
    return (G **2 / (H + lambda_)) * (-0.5)

In [5]:
def splitDataSet(dataSet, feat_index, feat_val):
    left_index = dataSet[:, feat_index] <= feat_val
    right_index = dataSet[:, feat_index] > feat_val
    
    left_dataSet = dataSet[left_index]
    right_dataSet = dataSet[right_index]
    
    return left_dataSet, right_dataSet

In [6]:
def findBestSplit(dataSet, avaliable_feat, previous_trees):
    base_cost = calGain(dataSet, previous_trees)
    cur_max_gain = 0 ; best_split_feat = -1 ; best_split_val = -1
    for feat_index in avaliable_feat:
        feat_valueRange = np.array(list(set(dataSet[:, feat_index])))   #获取该特征所有非重复取值
        sorted_feat_valueRange = sorted(feat_valueRange)
        
        for j in range(len(sorted_feat_valueRange) - 1):
            split_val = (sorted_feat_valueRange[j] + sorted_feat_valueRange[j+1]) / 2
            left_dataSet, right_dataSet = splitDataSet(dataSet, feat_index, split_val)
            gain = base_cost - calGain(left_dataSet, previous_trees) - calGain(right_dataSet, previous_trees)

            if gain > cur_max_gain:
                cur_max_gain = gain
                best_split_feat = feat_index
                best_split_val = split_val
    
    if best_split_feat == -1:    #不进一步划分,损失函数最小
        return None, None, None
    
    avaliable_feat.remove(best_split_feat)    #去掉已经作为过划分特征的特征
    
    return best_split_feat, best_split_val, avaliable_feat

In [7]:
def leaf_func(subSet, previous_trees, lambda_=1):
    y_hat = XGBoost_predict(subSet[:, :-1], previous_trees)
    G = (y_hat - subSet[:, -1]).sum()
    H = subSet.shape[0]
    
    return -(G / (H + lambda_))

In [8]:
def createTree(dataSet, previous_trees, leaf_func=leaf_func, max_depth=5, avaliable_feat_list=None, num_in_leaf=5):
    tree_dict = {}
    avaliable_feat = [i for i in range(len(dataSet[0]) -1)]
    best_split_feat, best_split_val, avaliable_feat = findBestSplit(dataSet, avaliable_feat, previous_trees)  #best_split_feat是某特征的索引值
    
    if best_split_feat == None:    #如果不划分该节点时损失函数最小的话
        return leaf_func(dataSet, previous_trees)
    
    tree_dict['split_feat'] = best_split_feat
    tree_dict['split_val'] = best_split_val
    left_sub_dataSet, right_sub_dataSet = splitDataSet(dataSet, best_split_feat, best_split_val)
    
    if max_depth == 0:
        tree_dict['left'] = leaf_func(left_sub_dataSet, previous_trees)
        tree_dict['right'] = leaf_func(right_sub_dataSet, previous_trees)
        return tree_dict

    if len(left_sub_dataSet) < num_in_leaf:
        tree_dict['left'] = leaf_func(left_sub_dataSet, previous_trees)
    else:
        tree_dict['left'] = createTree(left_sub_dataSet, previous_trees, leaf_func, max_depth-1, avaliable_feat)
        
    if len(right_sub_dataSet) < num_in_leaf:
        tree_dict['right'] = leaf_func(right_sub_dataSet, previous_trees)
    else:
        tree_dict['right'] = createTree(right_sub_dataSet, previous_trees, leaf_func,  max_depth-1, avaliable_feat)
        
    return tree_dict

In [9]:
def buildXGBoost(dataSet, num_tree=5):
    tree_list = [{'split_feat':0, 'split_val': 1e8, 'left':0}]
    for i in range(num_tree):
        tree_list.append(createTree(dataSet, tree_list.copy()))   #需要利用已有的树对样本的预测值
    
    return tree_list